# NVSS Source Counts

In this tutorial we will load the raw NVSS catalogue into python, have a look at the data and plot the Euclidean normalised source counts from this data.

The NVSS is a large area 1.4 GHz survey undertaken with the VLA. It covers the entire sky north of declination -40 degrees and has a limiting flux density of ~2.5 mJy.

The NVSS catalogue that we will be using in this tutorial has been constructed by finding all of the peaks above 2.5 mJy in the images and fitting a 2 dimensional elliptical Gaussian model to each source. The parameters of the elliptical Gaussian fit are:

1) The x,y position of the source (RA and Dec in our case)

2) The peak of the gaussian (in Jy/beam) - beam is the word used by radio astronomers for the resolution.

3) The 2D elliptical parameters of the source - Major Axis, Minor Axis, Position Angle

To start with we will import some required modules and set up some useful initial variables. We will use the module pyfits to load the NVSS catalogue, numpy for array manipulation and matplotlib to do our plotting. This tutorial assumes you have access to the NVSS catalogue in FITS binary table format (this is the standard data product produced from the NVSS survey). If you don't have the file you can download it from the NVSS webpage at http://www.cv.nrao.edu/nvss/ - this page also contains lots of information about the survey with which you may want to familiarse yourself.

NVSSFILE is a string containing the disk location of the NVSS catalogue fits file.
NVSSBEAM is the size of the NVSS resolution element (known as the 'beam' in radio astronomy) in degrees. The NVSS beam is a circular gaussian of width 45 arcseconds, so the width in degrees is 45/3600 (3600 arcseconds in a degree).

In [ ]:
import pyfits
import numpy as np
from matplotlib import pylab as plt
%matplotlib inline

NVSSFILE=''
NVSSBEAM=45./3600.

Now we will open the NVSS catalogue using pyfits and look at its structure. The fileptr=pyfits.open(<FILE>) command will load the file into a filepointer.

Fits files are separated into "Header/Data Units" or HDU's. The catalogue data is located in the second HDU in the file (the first just contains general header information about the catalogue).

The catalogue data contained in the second hdu, contains a list of sources with various parameters of each source. These parameters are derived from fits of gaussians at the position of each NVSS source in the survey. To list the names of the fitted parameters in the colums you should open the table (using nvsstable=fileptr[1].data) and then print the names of the columns using nvsstable.names.

In [ ]:
#Open the FITS file
fileptr   = pyfits.open(NVSSFILE)
#The NVSS catalogue is in the second HDU and is accesed via the 'data' attribute in the pyfits HDU object
nvsstable = fileptr[1].data
#Print the names of the columns in the data table



Here are some definitions of the columns of the table:
    
    RA(2000): The Right Ascension of the source
    DEC(2000): The declination of the source
    PEAK INT: The peak flux density of the Gaussian fitted to the source
    MAJOR AX: The major axis size of the fitted elliptical Gaussian
    MINOR AX: The minor axis size of the fitted elliptical Gaussian
    POSANGLE: The position angle of the fitted elliptical Gaussian
        
The other columns are not relevant to the tutorial. If you want to find out more about them, check the NVSS website.

Lets extract the required parameters from the catalogue to do the tutorial. I've filled in the first two, you need to add the rest from the list above.

In [ ]:
decs = nvsstable['DEC(2000)']    #Declination
ras = nvsstable['RA(2000)']      #Right Ascension
peakflux =                       #Fitted peak flux density                
majsize =                        #Fitted major axis size
minsize =                        #Fitted minor axis size
sourcepa =                       #Fitted position angle

Now before we do anything else we need to make one important correction.

The flux densities quoted in the raw NVSS catalogue we are using are fitted peak flux densities. We will need to convert the fitted peak flux densities into total flux densities before we can plot the source counts. To calculate the total flux density we just need to multiply the peak flux density by the area of the source. Peak flux densities are measured in units of mJy/beam and total flux density is just in units of mJy, so the area of the source we use should be in units of the beam. The sources in the NVSS catalogue are fitted with an elliptical Gaussian model. The beam of the NVSS (which we defined at the beginning of this tutorial) is also an elliptical Gaussian. We therefore just need to multiply the peak flux densities by the areas in units of the NVSS beam. This should be: totalflux=peakflux x (majsize x minsize) / (NVSSBEAM x NVSSBEAM)

Fill in the correct equation for total flux below.

In [ ]:
totalflux  = 

Now we are ready to make some plots.

First lets plot the distribution of the NVSS catalogue sources on the sky. The code in the block below should do this for you assuming the variables are set up correctly in the previous code blocks. Try and understand what it is doing.

If this code works you should see the NVSS sources above the chosen flux density limit plotted in a Mollweide projection - this projection corrects from the spherical sky to the flat computer screen. Note the isotropy of the radio sources away from the galactic plane.

The minimum flux plotted is set to 1 Jy, try setting this to 100 mJy to see what difference it makes.

In [ ]:
FLUXLIMJY = 1                  #Limiting flux for sources to plot.

# Correct the Right Ascensions to work with matplotlib mollweide projection and convert from degrees to radians.
# Subtract 180 deg. as matplotlib plots between -180 and 180 (RA is 0 to 360), 
# and multiply by -1 as RA runs from east to west.
plot_ras = (ras[totalflux>FLUXLIMJY]-180.)*np.pi/180.*-1.0
# Convert declination to radians
plot_decs = (decs[totalflux>FLUXLIMJY])*np.pi/180.

# Set up the figure
figure = plt.figure(figsize=(50,50))
ax = plt.subplot(111, projection='mollweide')
# Dont plot the RA labels (as number dont refect real RA's)
ax.xaxis.set_visible(False)
#Plot
ax.plot(plot_ras,plot_decs,'.')
plot.show()
plot.close()

Below is a code block to explore the NVSS catalogue yourself.

How many sources are in the catalogue?             (use: <array>.shape)

What is the minumum flux density of the catalogue? (use: np.min)

What is the maximum flux density of the catalogue? (use: np.max)

In [ ]:
numsources = 
minflux = 
maxflux = 
print numsources,minflux,maxflux

OK, now we will move to calculating the normalised source counts. You will largely be on your own here and the code blocks are mostly empty except some additions for setting up the plots.

In the code block below I have set up an array of logarithmically spaced bins to use for the counts using np.logspace, I have set up 50 bins from the minumum flux of the NVSS catalogue (defined in the previous code block as minflux) to 10 Jy - above this flux density the numbers of sources are very small and bright Galactic sources begin to dominate the population.

In the code block below you should calculate 3 things from the bins I have set up for you:

widths: The linear widths of the bins (ie. bins[i+1]-bins[i])

centres: The centres of the bins (ie. (bins[i] + bins[i+1])/2)

counts: The number of NVSS catalogue sources in each bin (np.histogram should be useful here- look it up online)

I have added a comment about errors- these should be the Poisson error in each bin (ie: sqrt(counts)) - you don't need to compute these now- but you can come back and do this later if you have time.

Aside: For the python-savvy amongst you - can you compute the variables in one line of code each? 

In [ ]:
# Set up a list of 50 logarithically spaced bins from 'minflux' to 10 Jy (logspace takes log10(range) as input)
bins = np.logspace(np.log10(minflux),1.0,50)

widths = 
centres = 
counts = 
# Poisson Errors??

OK, now that you have set up the required variables, first lets try plotting the raw numbers of NVSS sources as a function of their flux density. (ie - plot counts vs centres )

You should see that the raw numbers decline steeply with increasing flux density. 

Do you see a turnover at the faintest flux densities? Why might this be?

In [ ]:
#Set up a log-log plot for N as a function of S
figure=plt.figure(figsize=(15,10))
plt.ylabel("N")
plt.xlabel("S (Jy)")
plt.yscale('log')
plt.xscale('log')



Now that you have plotted the raw value of N as a function of s. The next step is to plot the differential function n(s) (or dn/ds). This is just the raw count N divided by the bin width.

Go ahead and to plot n(s)=count/width vs totalflux.

In [ ]:
#Calculate n(s)

# Set up a log-log plot for n(s) as a function of S
figure=plt.figure(figsize=(15,10))
plt.ylabel("n(s)")
plt.xlabel("S (Jy)")
plt.yscale('log')
plt.xscale('log')


One of the reasons for the steepness seen in the unnormised counts you have plotted is that the number of sources is very large in the volume of space we a sampling- we need to make a euclidean correction to account for this.

We will plot the Euclidean normalised count, as was seen in the lectures. 

Remember the Euclidean normalisation flattens the n(s) counts for a static Euclidean universe:

Assuming a class of uniformly distributed objects with a luminosity L which are detectable some limiting flux density, S, out to a distance r. The observed number, N is then proportional to the Volume: 

$N \propto r^3$

The flux density observed from a source at a distance r follows the inverse-square law: 

$S \propto r^{-2}$

therefore:

$r^3 \propto S^{-3/2}$

and combining things gives:

$N(>S) \propto S^{-3/2}$

and the differential version is:

$n(s) \propto S^{-5/2}$

Therefore, to normalise the above we need to divide our counts from the previous plot (n(s)) , by $S^{-5/2}$

Try this now and plot it below.

How does this compare with the counts shown in lectures? Can you explain any differences?

If you have time- perhaps you can add poisson error bars to this plot?

In [ ]:
#Calculate the normalised Euclidean count

#Set up Figure for plotting
figure=plt.figure(figsize=(15,10))
plt.ylabel("n(s) / S^-2.5")
plt.xlabel("S (Jy)")
plt.yscale('log')
plt.xscale('log')
